In [1]:
%pip install tqdm

     -------------------------------------- 78.4/78.4 kB 725.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from card_game import CardGame, Player, Card
import random

player = 1

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class RandomPlayer(Player):
    """
    Makes random moves (but according to the rules)
    """
    def __init__(self):
        global player
        self.number = player
        player += 1

    def make_move(self, game_state: dict, was_previous_move_wrong: bool) -> Card:
        if not game_state["discard"]:
            return random.choice(game_state["hand"])
        else:
            options = list(filter(lambda card: card.suit == list(game_state["discard"])[0].suit, game_state["hand"]))
            if len(options) > 0:
                return random.choice(options)
            else:
                return random.choice(game_state["hand"])

    def get_name(self):
        return f"RandomPlayer{self.number}"

    def set_temp_reward(self, discarded_cards: dict, point_deltas: dict):
        pass

    def set_final_reward(self, points: dict):
        pass


In [3]:
from agents import REINFORCEAgent
game = CardGame(REINFORCEAgent(32, False, 1e-3, layers=[256, 128]), RandomPlayer(), RandomPlayer(), RandomPlayer(), full_deck=False, display=False)

72 24


In [4]:
game.players[0]

REINFORCEAgent(
  (qnet): Sequential(
    (0): Sequential(
      (0): Linear(in_features=72, out_features=256, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=128, out_features=24, bias=True)
      (1): Identity()
    )
  )
)

In [5]:
game.start()

72


{REINFORCEAgent(
   (qnet): Sequential(
     (0): Sequential(
       (0): Linear(in_features=72, out_features=256, bias=True)
       (1): ReLU()
     )
     (1): Sequential(
       (0): Linear(in_features=256, out_features=128, bias=True)
       (1): ReLU()
     )
     (2): Sequential(
       (0): Linear(in_features=128, out_features=24, bias=True)
       (1): Identity()
     )
   )
 ): 41,
 <__main__.RandomPlayer at 0x22275e404c0>: 41,
 <__main__.RandomPlayer at 0x22212e507c0>: 99,
 <__main__.RandomPlayer at 0x22212e50760>: 28}